In [1]:
import sys
import os
base_dir = "/home/jingpei/Desktop/CtRNet-robot-pose-estimation"
sys.path.append(base_dir)

import torch
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import numpy as np
import matplotlib.pyplot as plt

from utils import *
from models.CtRNet import CtRNet
from models.BPnP import batch_transform_3d
from evaluate import evaluate_ADD


In [2]:
import argparse
from imageloaders.DREAM import ImageDataLoaderReal, load_camera_parameters
parser = argparse.ArgumentParser()

args = parser.parse_args("")

args.data_folder = "/media/jingpei/DATA/DREAM/data/real/panda-3cam_realsense"
args.base_dir = "/home/jingpei/Desktop/CtRNet-robot-pose-estimation"
args.use_gpu = True
args.trained_on_multi_gpus = True
args.keypoint_seg_model_path = os.path.join(args.base_dir,"outputs/pretrain/net_epoch_3.pth")
args.urdf_file = os.path.join(args.base_dir,"urdfs/Panda/panda.urdf")

args.robot_name = 'Panda' # "Panda" or "Baxter_left_arm"
args.n_kp = 12
args.scale = 0.5
args.height = 480
args.width = 640
args.fx, args.fy, args.px, args.py = load_camera_parameters(args.data_folder)

# scale the camera parameters
args.width = int(args.width * args.scale)
args.height = int(args.height * args.scale)
args.fx = args.fx * args.scale
args.fy = args.fy * args.scale
args.px = args.px * args.scale
args.py = args.py * args.scale

In [3]:
CtRNet = CtRNet(args)

Loading keypoint segmentation model from /home/jingpei/Desktop/CtRNet-robot-pose-estimation/outputs/pretrain/net_epoch_3.pth
Camera intrinsics: [[307.76196289   0.         164.13032532]
 [  0.         307.60958862 125.89585114]
 [  0.           0.           1.        ]]
Robot model: Panda


In [4]:
trans_to_tensor = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

dataset = ImageDataLoaderReal(data_folder = args.data_folder, scale = args.scale, trans_to_tensor = trans_to_tensor)

In [5]:
AUC, mean_error = evaluate_ADD(CtRNet, dataset)

  0%|          | 0/5944 [00:00<?, ?it/s]

In [7]:
print("AUC: ", AUC)
print("Mean error: ", mean_error)

AUC:  0.7832614401076716
Mean error:  0.022072875540795213
